## Optimizing Resonator Designs

There are many factors for the resonator design. I categorized the factors into two tiers:

Top Tier:
1. Length
2. Center pin to gap ratio (this defines the impedance of the resonator). 

Second Tier:
1. Center pin width


In [26]:
import astropy.units as u
import astropy.constants as c
import scipy.optimize as optimize

In [41]:
# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 250*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width

### Use Qiskit-metal function to calculate transmission line pin/gap ratio

Note to self: when constructing function for scipy, the variable to optimize has to be dimensionless. 

In [42]:
import qiskit_metal.analyses as analyses

In [43]:
def f(line_gap):
    return(analyses.cpw_calculations.lumped_cpw(freq.to(u.Hz).value,
                                     line_width.to(u.m).value,
                                     line_gap,
                                     sub_t.to(u.m).value,
                                     metal_t.to(u.m).value)[4]-50)

In [44]:
freq = 7*u.GHz.si
line_width = 15.322*u.um
line_gap = 10*u.um

In [27]:
res = analyses.cpw_calculations.lumped_cpw(freq.to(u.Hz).value,
                                     line_width.to(u.m).value,
                                     line_gap.to(u.m).value,
                                     sub_t.to(u.m).value,
                                     metal_t.to(u.m).value)

Z0 = res[4]*u.Ohm

In [28]:
Z0

<Quantity 52.72897025 Ohm>

In [56]:
optimized_gap = optimize.fsolve(f,line_gap.si.value)[0]*u.m

In [57]:
optimzied_gap

<Quantity 8.28349609 um>

In [58]:
f(optimized_gap.si.value)

7.105427357601002e-15

In [60]:
analyses.cpw_calculations.lumped_cpw(freq.to(u.Hz).value,
                                     line_width.to(u.m).value,
                                     optimized_gap.to(u.m).value,
                                     sub_t.to(u.m).value,
                                     metal_t.to(u.m).value)

(1.6655452372229007e-09,
 4.211256177953425e-07,
 1.6845024711813694e-10,
 3.703063394264467e-05,
 50.00000000000001,
 6.082601695790244,
 9.583285084677287e-11)

### Using the optimzied pin/gap ratio optimzie to construct a python function that takes coupling strength between and outputs a coupled line Tee design. 

In [1]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [4]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('Customized_Components')
from single_pad_transmon_pocket import TransmonPocket_Single

import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [2]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

In [147]:
pocket_width = 350*u.um
CPW_pocket_extend = 5*u.um

In [148]:
str(pocket_width)

'350.0 um'

In [163]:
design.overwrite_enabled = True #This makes it possible to edit the designs later on. 

design.delete_all_components()

options = dict(pad_width = '330um',
pocket_width = '{}'.format(pocket_width),
pad_pocket_distance_top = '30um',
pos_x = '0',
pad_height = '260um',
pos_y = '0',
connection_pads = dict(
    a = dict(loc_W  = 1, 
             loc_H =  1, 
             pad_gap = '10um',
             pad_height = '15um',
             pad_width = '60um',
             pad_cpw_extent =  '10um',
             pocket_rise = '0um',
             cpw_extend = '{}'.format(CPW_pocket_extend))))

q1 = TransmonPocket_Single(design,'Q1',options = options)

gui.rebuild()
gui.zoom_on_components(['Q1'])

In [164]:
#This is a function to generate needed pins for CPW drawing. 
import numpy as np
from collections import OrderedDict


def pins_for_spiral(diameter,gap,n,dis = (0,0)):
    spiral_list = OrderedDict()
    x,y = dis
    for step in range(n):
        point_value = diameter / 2 + step * (0 + gap)
        spiral_list[step*4] = np.array([x+point_value, y-point_value])
        spiral_list[step*4+1] = np.array([x-point_value, y-point_value])
        spiral_list[step*4+2] = np.array([x-point_value, y+point_value])
        spiral_list[step*4+3] = np.array([x+point_value + (0 + gap), y+point_value])

    point_value = diameter / 2 + (step + 1) * (0 + gap)
    # spiral_list[step*4+4] = np.array([-point_value, -point_value])
    return(spiral_list)

In [170]:
diameter = (pocket_width+CPW_pocket_extend*2).to(u.mm).value
gap = 0.04
diameter += gap

offset = 0.025

anchors = pins_for_spiral(diameter, gap,2,dis = (0,0-offset))
print(anchors)


OrderedDict([(0, array([ 0.2  , -0.225])), (1, array([-0.2  , -0.225])), (2, array([-0.2  ,  0.175])), (3, array([0.24 , 0.175])), (4, array([ 0.24 , -0.265])), (5, array([-0.24 , -0.265])), (6, array([-0.24 ,  0.215])), (7, array([0.28 , 0.215]))])


In [171]:
options = Dict(total_length='4 mm',
        hfss_wire_bonds = True,
        q3d_wire_bonds = True,
        fillet='20 um',
        lead = dict(start_straight='20um', end_straight = '0um'),
        anchors = anchors,
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='end', pin='short')), )

In [172]:
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed

In [173]:
stg = ShortToGround(design,'end',options = dict(pos_x = '0.5mm',pos_y = '0.5mm',orientation = '90'))
gui.rebuild()
gui.highlight_components(['end'])

In [174]:
qa = RouteMixed(design, 'cpw_1', options)

gui.rebuild()
gui.autoscale()

In [175]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [177]:
eig_res = EPRanalysis(design, "hfss")

hfss = eig_res.sim.renderer
hfss.start()

hfss.activate_ansys_design('readout','eigenmode')

hfss.render_design(['cpw_1','Q1','end'],[])
hfss.save_screenshot()

setup = hfss.pinfo.setup
setup.passes = 40
setup.min_converged = 9
setup.n_modes = 2

print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

# Next 2 lines are counterinuitive, since there is no junction in this resonator.
# However, these are necessary to make pyEPR work correctly. Please do note delete
hfss.pinfo.design.set_variable('Lj', '0 nH')
hfss.pinfo.design.set_variable('Cj', '0 fF')

setup.analyze()

INFO 11:30PM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:30PM [load_ansys_project]: 	Opened Ansys App
INFO 11:30PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 11:30PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/slab/Documents/Ansoft/
	Project:   Project15
INFO 11:30PM [connect_design]: No active design found (or error getting active design).
INFO 11:30PM [connect]: 	 Connected to project "Project15". No design detected
11:30PM 32s WARNING [activate_ansys_design]: The design_name=readout was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 11:30PM [connect_design]: 	Opened active design
	Design:    readout [Solution type: Eigenmode]
WARNING 11:30PM [connect_setup]: 	No design setup detected.
WARNING 11:30PM [connect_setup]: 	Creating eigenmode default setup.
INFO 11:30PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
11:30PM 36s ERROR [subtract_fro

com_error:  (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024381), None)


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024381), None)